## LLM as a Research Assisstant


In [30]:
import os
from dotenv import load_dotenv
load_dotenv()

python-dotenv could not parse statement starting at line 2


True

In [31]:
# Install core PDF and document processing dependencies
"""!pip install pdfminer.six
!pip install "unstructured[local-inference]"==0.10.25  # or latest compatible with langchain
!pip install tesseract
!pip install poppler

# LangChain and related dependencies
!pip install langchain langchain-community langchain-openai langchain-core langchain-text-splitters

"""
#pip install pdfminer
#pip install pdfminer.six
#!pip install "unstructured[local-inference]" pdfminer.six==20221105
#pip install tesseract
#pip install tesseract poppler

'!pip install pdfminer.six\n!pip install "unstructured[local-inference]"==0.10.25  # or latest compatible with langchain\n!pip install tesseract\n!pip install poppler\n\n# LangChain and related dependencies\n!pip install langchain langchain-community langchain-openai langchain-core langchain-text-splitters\n\n'

In [32]:
from langchain_community.document_loaders import UnstructuredPDFLoader

file_path = "mindset2.pdf"
loader = UnstructuredPDFLoader(file_path)
docs = loader.load()

In [33]:
# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

document_ids = vector_store.add_documents(documents=all_splits)

In [34]:
all_splits

[Document(metadata={'source': 'mindset2.pdf', 'start_index': 0}, page_content='OPEN ACCESS\n\nEDITED BY Yajun Zhao, Southwest Minzu University, China\n\nREVIEWED BY Jianchao Lai, University of California, Los Angeles, United States Xiaochun Zhou, China Youth University for Political Sciences, China\n\nCORRESPONDENCE Zhongfang Fu z.fu@pku.edu.cn\n\nRECEIVED 31 October 2024 ACCEPTED 20 January 2025 PUBLISHED 12 February 2025\n\nCITATION Fang Z and Fu Z (2025) Exploring the impact of growth mindset on psychological symptoms in students from ethnic regions of China: how meaning in life makes a difference. Front. Psychiatry 16:1520645. doi: 10.3389/fpsyt.2025.1520645'),
 Document(metadata={'source': 'mindset2.pdf', 'start_index': 583}, page_content='COPYRIGHT © 2025 Fang and Fu. This is an open-access article distributed under the terms of the Creative Commons Attribution License (CC BY). The use, distribution or reproduction in other forums is permitted, provided the original author(s) and

In [35]:
question="What are the key ideas from this research article?"

In [36]:
# extract chunks which matches with your query
search_results = vector_store.similarity_search_with_score(question, k=9)

# conbine text from differrent chunks into one long string
doc_content = "\n\n".join(doc.page_content for (doc,score) in search_results)

In [37]:

prompt_template = """You are an assistant to a Psychologist Use the following pieces of retrieved 
context to answer the question. Use no data outside of the context. If you don't know the answer,
 just say that you don't know, don't try to make up an answer. AND if the question is not related 
 to the context, politely respond that you are tuned to only answer questions that are related 
 to the context. Make sure the answer is concise and not too long. Also make sure to include examples
 wherever possible.
Question: {question} 
Context: {context} 
Answer:"""

from langchain.chat_models import init_chat_model
llm = init_chat_model("openai/gpt-oss-20b", model_provider="groq")

response = llm.invoke(prompt_template.format(
    context=doc_content,
    question="What are the key ideas from this research article?"))
    
from IPython.display import Markdown
Markdown(response.content)

**Key ideas from the article**

1. **Growth mindset protects mental health** – Students who believe abilities can improve tend to show fewer psychological symptoms.  
   *Example:* A student who sees effort as a path to success reports lower anxiety than a student who thinks talent is fixed.

2. **Meaning in life mediates this protection** – The benefit of a growth mindset on mental health operates through a stronger sense of meaning.  
   *Example:* A growth‑mindset student who finds purpose in schoolwork reports fewer depressive symptoms, whereas the same student without a sense of meaning does not see this benefit.

3. **The mediation holds over time** – The cross‑sectional and longitudinal models both show the same pattern, indicating that the effect is stable from one time point to the next.  
   *Example:* Over a school year, growth‑mindset students who maintain a sense of meaning continue to show reduced stress.

4. **Practical implication for schools** – Interventions that cultivate growth mindset and help students build meaning can yield lasting mental‑health benefits, especially for disadvantaged adolescents.  
   *Example:* A middle‑school program that combines mindset workshops with reflective journaling on personal goals could reduce student‑reported psychological symptoms.

5. **Future research directions** – Need for culturally adapted tools, more time points, and randomized designs to confirm causality.  
   *Example:* A randomized controlled trial of a growth‑mindset curriculum in an ethnic minority region could test whether the observed mediation is causal.

In [38]:
response = llm.invoke(prompt_template.format(
    context=doc_content,
    question="What is the net worth of donald trump?"))
    
from IPython.display import Markdown
Markdown(response.content)

I’m sorry, but I’m only able to answer questions that relate to the provided context.

In [39]:
question="""If someone wants to pursue research in psychology, what are the key 
    ideas from this research article that they should focus on?"""
search_results = vector_store.similarity_search_with_score(question, k=3)
doc_content = "\n\n".join(doc.page_content for (doc,score) in search_results)

In [40]:
prompt_template = """A Psychology undergrad  student needs help in understanding a research article,
help the student by answering the following question. Use the following pieces of retrieved 
context to answer the question. Use no data outside of the context. 
Question: {question} 
Context: {context} 
Answer:"""

response = llm.invoke(prompt_template.format(
    context=doc_content,
    question="""If someone wants to pursue research in psychology, what are the key 
    ideas from this research article that they should focus on?"""))
    
from IPython.display import Markdown
Markdown(response.content)

**Key ideas to focus on when pursuing research in psychology (based on the article):**

1. **Causality vs. Correlation**  
   - The study demonstrates that while a longitudinal design can establish temporal order, it cannot definitively prove that growth mindset *causes* changes in mental health.  
   - Future work should therefore employ **randomized controlled designs** or **targeted interventions** to strengthen causal claims.

2. **Growth Mindset as a Variable of Interest**  
   - Growth mindset is central to the model; it is linked to psychological symptoms through mediating mechanisms.  
   - Researchers should consider how to **measure, manipulate, and intervene** on growth mindset in experimental settings.

3. **Mediation by Meaning in Life**  
   - The article highlights **meaning in life** as a key mediator between growth mindset and psychological symptoms.  
   - Investigating the pathways and mechanisms of this mediation (e.g., how meaning is constructed, maintained, or disrupted) is essential.

4. **Cultural and Contextual Adaptation**  
   - The sample came from ethnic regions of China, underscoring the need for **culturally adapted tools** to improve both generalizability and relevance.  
   - Researchers should develop or validate instruments that reflect local cultural nuances.

5. **Developmental (Age) Considerations**  
   - Differences in average age across samples suggested developmental stages may influence the relationships studied.  
   - Future research should **explore age‑specific mechanisms** to design interventions that are developmentally appropriate.

6. **Generalizability and Relevance**  
   - The authors note that findings are consistent across mediation paths but that cultural and age factors may moderate effects.  
   - Emphasizing **external validity** (e.g., diverse samples, cross‑cultural replication) will strengthen the field.

7. **Methodological Rigor**  
   - The limitations point to the importance of **experimental manipulation** and **random assignment** to control for confounding variables.  
   - Researchers should plan studies that can isolate the effect of growth mindset from other influences.

**Take‑away for a psychology researcher:**  
Focus on designing experiments that manipulate growth mindset, measure meaning in life, and assess mental health outcomes across culturally diverse and age‑specific populations. Such work will help clarify causal mechanisms, improve intervention relevance, and enhance the field’s overall robustness.

In [50]:
question="""placebo"""
search_results = vector_store.similarity_search_with_score(question, k=3)
doc_content = "\n\n".join(doc.page_content for (doc,score) in search_results)

In [51]:

response = llm.invoke(prompt_template.format(
    context=doc_content,
    question="Does this article discuss the impact of mindset with regards to placebo effect?"))
    
from IPython.display import Markdown
Markdown(response.content)

No. The excerpts provided focus on digital‑detox coping attempts, the relationship between doom scrolling and mental well‑being, demographic data, and the overall study methodology. There is no mention of “mindset” or any discussion of the placebo effect in the article.

In [52]:
question="""impressive and new findings,conclusions"""
search_results = vector_store.similarity_search_with_score(question, k=3)
doc_content = "\n\n".join(doc.page_content for (doc,score) in search_results)
response = llm.invoke(prompt_template.format(
    context=doc_content,
    question="""What is the thing from this article that will impress my psychology prof the 
    most if i tell it to her?"""))
    
from IPython.display import Markdown
Markdown(response.content)

The most striking point that will catch your professor’s eye is the **statistically significant, strong positive correlation** found in the study:  

- **Pearson’s r = 0.672** between doom‑scrolling and emotional exhaustion, with a **p‑value of 0.00** (well below the 0.05 threshold).  
- This result led to a clear rejection of the null hypothesis, confirming that doom scrolling has a *considerable* negative effect on the mental health of media students in Karachi.

Highlighting this precise, high‑correlation statistic shows the study’s rigorous quantitative evidence, which is sure to impress.

In [43]:
file_path = "doomscroll1.pdf"
loader2 = UnstructuredPDFLoader(file_path)
docs2 = loader2.load()

all_splits2 = text_splitter.split_documents(docs2)
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector_store = InMemoryVectorStore(embeddings)
document_ids2 = vector_store.add_documents(documents=all_splits2)
all_splits2

[Document(metadata={'source': 'doomscroll1.pdf', 'start_index': 0}, page_content='P-ISSN: 2790-6795 O-ISSN:2790-6809\n\nAnnals of Human and Social Sciences\n\nhttp://doi.org/10.35484/ahss.2025(6-II)30\n\nApril-June 2025,Vol. 6, No. 2\n\n[357-369]\n\n25 , w/L\n\nRESEARCH PAPER\n\nImpact of Doom Scrolling on Mental Well-being among Media Students in Karachi\n\n1Dr. Sobia Usman*, 2 Tazeen Huma and 3Muhmmad Rashid Akbar\n\n1. Associate Professor, Media Sciences Department, Faculty of Media Sciences, Iqra University Karachi, Sindh, Pakistan\n\n2. PhD Scholar & Senior Lecturer, Media Sciences Department, Faculty of Media Sciences, Iqra University Karachi, Sindh, Pakistan\n\n3. PhD Scholar & Assistant Professor, Media Sciences Department, Faculty of Media Sciences, Iqra University Karachi, Sindh, Pakistan'),
 Document(metadata={'source': 'doomscroll1.pdf', 'start_index': 707}, page_content='Corresponding Author ABSTRACT This study assess the psychological effects of doom scrolling on the ment

In [69]:
question="doomscrolling definition, why is it so common nowadays"
search_results2 = vector_store.similarity_search_with_score(question, k=1)
doc_content2 = "\n\n".join(doc.page_content for (doc,score) in search_results2)

In [66]:

prompt_template2 = """You are a research assistant for a school principal who has to give a 
speech on the adverse effects of doomscrolling, use the following pieces of context to answer
her questions. be concise and elaborate where necessary. answer solely based on the context
NO OUTSIDE INFO SHOULD BE USED.
Question: {question} 
Context: {context} 
Answer:"""

response = llm.invoke(prompt_template2.format(
    context=doc_content2,
    question="What is doomscrolling, why is it so common nowadays "))
    
from IPython.display import Markdown
Markdown(response.content)

**Doomscrolling** is the habit of repeatedly scrolling through negative news or distressing content online, often as a coping mechanism that ultimately backfires (McGowan, 2020).  

**Why it’s become so common today**

| Factor | Explanation |
|--------|-------------|
| **Pandemic acceleration** | The COVID‑19 crisis intensified doomscrolling, prompting Park et al. (2020) to warn that the habit was spreading rapidly and calling for institutional intervention. |
| **Mental‑health and media overload** | Constant exposure to bad news raises stress and hampers clear thinking and problem‑solving, making the behavior self‑reinforcing (McGowan, 2020). |
| **Cyberchondria link** | Wiederhold (2020) linked doomscrolling to health anxiety, where obsessive online searching fuels worry; media literacy is key to breaking this loop. |
| **Digital culture** | The ease of accessing endless streams of content, combined with algorithmic amplification of sensational stories, makes doomscrolling almost unavoidable without conscious effort. |

In short, doomscrolling is a compulsive, negative‑news‑driven coping strategy that has surged during the pandemic and is sustained by the interplay of stress, health anxiety, and the design of online platforms. Institutions are urged to blend mental‑health resources with digital literacy training to help users recognize and curb these harmful patterns.

In [ ]:
question="Effect of doomscrolling on attention span,concentration, productivity"
search_results2 = vector_store.similarity_search_with_score(question, k=1)
doc_content2 = "\n\n".join(doc.page_content for (doc,score) in search_results2)

In [76]:


response = llm.invoke(prompt_template2.format(
    context=doc_content2,
    question="Does it shorten our attention span?"))
    
from IPython.display import Markdown
Markdown(response.content)

**Answer**

The excerpt you provided outlines the study’s methodology and indicates that the results and discussion sections will link the findings to the study’s objectives and theoretical framework (Uses and Gratifications Theory and Social Comparison Theory). However, the specific outcome regarding whether doomscrolling shortens our attention span is **not** included in the text you shared. Therefore, based solely on the provided context, we cannot confirm whether the study found a shortening of attention span.

In [47]:
response = llm.invoke(prompt_template2.format(
    context=doc_content2,
    question="Draft me a short speech to give on founders day "))
    
from IPython.display import Markdown
Markdown(response.content)

**Founders Day Speech – The Hidden Cost of Doomscrolling**

Good morning, distinguished guests, faculty, students, and fellow alumni.  
Today, as we celebrate the spirit of innovation and resilience that our founders instilled in us, I would like to bring your attention to a silent threat that challenges the very well‑being of our students—doomscrolling.

**What is doomscrolling?**  
In our digital age, students often turn to social media for information, connection, and emotional regulation—what the Uses and Gratifications Theory (UGT) calls “emotional and informational needs.” Yet, the content they consume is frequently distressing, especially during uncertain times.

**Why does it matter?**  
Our recent mixed‑method study of media students in Karachi revealed:

- **More than two hours of doomscrolling daily** for the majority of participants.  
- **Strong correlations** between prolonged doomscrolling and symptoms of **anxiety, emotional fatigue, and poor sleep**.  
- Qualitative interviews confirmed that students feel **inadequate and anxious** after comparing themselves to the idealized success or crisis‑handling of peers—a phenomenon explained by Social Comparison Theory.

**The psychological toll**  
When students continuously compare themselves to others’ curated lives, they experience heightened emotional turmoil. The constant influx of distressing news fuels anxiety and depression, while the urge to stay updated undermines sleep and concentration.

**Our responsibility**  
The study highlights three critical gaps:

1. **Lack of awareness** about the psychological effects of doomscrolling.  
2. **Underutilization of structured coping mechanisms.**  
3. **Need for institutional digital‑hygiene education** at the curriculum level.

**What we can do**  
- **Integrate digital‑wellness modules** into our media curriculum, teaching students how to curate their feeds and set healthy boundaries.  
- **Promote coping strategies**—mindfulness, scheduled breaks, and offline activities—to mitigate emotional fatigue.  
- **Encourage reflective discussions** that challenge the unrealistic standards often portrayed online.

**In closing**  
Our founders envisioned a community that thrives on knowledge, creativity, and well‑being. By acknowledging and addressing the adverse effects of doomscrolling, we honor that legacy. Let us equip our students with the tools to navigate the digital world responsibly, ensuring that their curiosity and ambition are matched by resilience and mental health.

Thank you.

In [48]:
response = llm.invoke(prompt_template2.format(
    context=doc_content2,
    question="What are the adverse effects of doomscrolling and after how much time do they start showing "))
    
from IPython.display import Markdown
Markdown(response.content)

**Adverse effects of doomscrolling**

- **Anxiety** – persistent worry and nervousness that rises with continual exposure to distressing news.  
- **Depression** – feelings of low mood and hopelessness that correlate with frequent doomscrolling.  
- **Emotional fatigue / burnout** – a sense of being drained or exhausted by the constant stream of negative content.  
- **Sleep disturbances** – trouble falling or staying asleep, often reported by students who scroll excessively.  
- **Concentration problems** – difficulty focusing on studies or other tasks after prolonged scrolling.  

**When the symptoms start to appear**

The study found that **most students engaged in doomscrolling for more than 2 hours each day**.  Strong statistical correlations were reported between this duration and the above psychological symptoms.  Thus, adverse effects tend to surface when the behavior extends beyond roughly **two hours of continuous scrolling per day**.

In [49]:

prompt_template2 = """A couple of parents want to make their 9 year old  kid aware of the adverse 
effects of doomscrolling, use the following pieces of context to answer their questions.Make sure
to use child friendly language and examples.
Question: {question} 
Context: {context} 
Answer:"""

response = llm.invoke(prompt_template2.format(
    context=doc_content2,
    question="What is doomscrolling, is it bad, who does it the most and why, should a 9 year old do it? "))
    
from IPython.display import Markdown
Markdown(response.content)

**Hey there!**  
Let’s talk about *doomscrolling* in a way that’s easy to understand and a bit like a story.

---

## 1. What is doomscrolling?  
- **Think of a “scroll”**: When you’re on a phone or tablet, you swipe or tap to see more pictures, videos, or text.  
- **Add “doom” to it**: Doomscrolling means you keep scrolling, but you’re only looking at *bad* or scary news—like big storms, fights, or other scary things.  
- **Why it’s called “doom”**: Because it’s all about “doom” (bad stuff) and can make you feel worried or upset.

*Example:* Imagine you’re looking at a news feed and every time you swipe you see another article about a new school rule, a sports injury, or a big storm. You keep swiping because you’re curious, but the more you see, the more worried you feel.

---

## 2. Is it bad?  
Yes, especially if you do it a lot.

| What the research says | What it means for you |
|------------------------|-----------------------|
| **Lots of students (like the ones in the study) spent more than 2 hours a day doomscrolling** | That’s a long time to be looking at scary news. |
| **Doomscrolling is linked to feeling anxious, tired, or having trouble sleeping** | You might feel on edge, have a hard time relaxing, or feel tired even after a good night’s sleep. |
| **It can make you compare yourself to others and feel less confident** | If you see others looking “cool” or “happy” on social media, you might think you’re not as good, even if that’s not true. |

So, yes—doomscrolling can make you feel *not* great.

---

## 3. Who does it the most and why?  
- **College and university students** (especially those studying media, like the Karachi study) were the biggest group in the research.  
- **Why they do it**:  
  - *Need to stay informed*: They want to know what’s happening in the world.  
  - *Feel connected*: They think looking at news keeps them part of a community.  
  - *Emotional regulation*: Sometimes they scroll to feel less alone when something scary happens.  
- **Other groups** (like teens and adults) also scroll a lot, but the study shows students are especially prone because they’re learning and planning their future.

---

## 4. Should a 9‑year‑old do it?  
**No, and here’s why:**

| Reason | Kid‑friendly explanation |
|--------|--------------------------|
| **You’re still growing** | Your brain is still learning how to handle big emotions. Constant scary news can make you feel scared or sad. |
| **You don’t need that much news** | At 9, you already hear enough from parents, teachers, and friends. Extra scary news isn’t helpful. |
| **It can affect sleep** | If you’re worried, it’s harder to fall asleep or dream sweetly. |
| **You’re better off doing fun things** | Reading a book, playing outside, drawing, or listening to music helps you feel happy and relaxed. |

**Bottom line:** It’s okay to learn about the world, but it’s best to do it in a balanced way—like reading a short news article once a week or talking with a parent about big events.

---

## 5. Quick “Digital Hygiene” Tips (for the whole family)

1. **Set a timer** – Only look at news for 10–15 minutes a day.  
2. **Mix in positive content** – Watch a funny cartoon or read a story that makes you smile.  
3. **Talk about feelings** – If something scary is bothering you, share it with a parent or teacher.  
4. **Take breaks** – Go outside, play a game, or do a craft.  
5. **Ask questions** – “Why is this happening?” helps you understand instead of just feeling scared.

---

### Final Thought

Doomscrolling is like reading a book that only has scary stories in it—over time it can make you feel tired, worried, and not as happy. For a 9‑year‑old, it’s best to focus on fun, learning, and staying connected with family, while keeping scary news to a minimum. If you ever feel upset after seeing something online, tell a trusted adult—there’s always help and a chance to feel better.